In [1]:
with open('fa_q.txt',encoding='UTF-8') as f:
    text = f.read()

In [2]:
text[:100]


"'  به نام خداوند بخشنده ى مهربان   سپاس و ستایش مخصوص خداوندى است که پروردگار جهانیان است   خدایى که"

In [3]:
len(text)

783774

In [4]:
len(text.split())

173587

# Word2Vec



# Step 0: Imports

In [5]:
import collections
import math
import os
import errno
import random

import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# Step 1: The data.

In [6]:
words = text.split()

In [7]:
# Total words
len(words)

173587

In [8]:
# Random slice of words
words[9000:9040]

['و',
 'میان',
 'آنها',
 'اختلافاتى',
 'پیدا',
 'شد',
 'پس',
 'خداوند',
 'پیامبران',
 'را',
 'بشارت',
 'آور',
 'و',
 'بیم',
 'دهنده',
 'برانگیخت',
 'و',
 'با',
 'آنان',
 'کتاب',
 'آسمانى',
 'را',
 'به',
 'حق',
 'نازل',
 'نمود',
 'تا',
 'میان',
 'مردم',
 'در',
 'آنچه',
 'اختلاف',
 'داشتند',
 'حکم',
 'کنند',
 'و',
 'تنها',
 'کسانى',
 'که',
 'کتاب']

In [9]:
for w in words[9000:9040]:
    print(w,end=' ')

و میان آنها اختلافاتى پیدا شد پس خداوند پیامبران را بشارت آور و بیم دهنده برانگیخت و با آنان کتاب آسمانى را به حق نازل نمود تا میان مردم در آنچه اختلاف داشتند حکم کنند و تنها کسانى که کتاب 

## Build Word Counts

In [10]:
from collections import Counter

In [11]:
mylist = ["one",'one','two']

In [12]:
Counter(mylist)

Counter({'one': 2, 'two': 1})

In [13]:
Counter(mylist).most_common(1)

[('one', 2)]

## Create Word Data and Vocab

In [14]:
def create_counts(vocab_size=5000):

    # Begin adding vocab counts with Counter
    vocab = [] + Counter(words).most_common(vocab_size )
    
    # Turn into a numpy array
    vocab = np.array([word for word, _ in vocab])
    
    
    dictionary = {word: code for code, word in enumerate(vocab)}
    data = np.array([dictionary.get(word, 0) for word in words])
    return data,vocab

In [15]:
vocab_size = 5000

In [16]:
# This may take awhile
data,vocabulary = create_counts(vocab_size=vocab_size)

In [17]:
data[1]

3

In [18]:
(words[100],data[100])

('دارند', 111)

In [19]:
vocabulary.shape

(5000,)

In [20]:
data.min()

0

In [21]:
data.max()

4999

In [22]:
vocabulary[np.random.randint(0,5000)]

'والا'

## Function for Batches

Direct from the word2vec file from TensorFlow Docs

In [23]:
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
    if data_index == len(data):
        buffer[:] = data[:span]
        data_index = span
    else:
        buffer.append(data[data_index])
        data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [24]:
data_index=0
batch, labels = generate_batch(8, 2, 1)

In [25]:
batch

array([3, 3, 3, 3, 3, 3, 3, 3])

In [26]:
labels

array([[ 352],
       [3924],
       [3924],
       [ 352],
       [ 352],
       [3924],
       [3924],
       [ 352]])

## Constants

In [27]:
# Size of the bath
batch_size = 128

# Dimension of embedding vector
embedding_size = 150

# How many words to consider left and right (the bigger, the longer the training)
skip_window = 1       

# How many times to reuse an input to generate a label
num_skips = 2        

In [28]:
# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.

# Random set of words to evaluate similarity on.
valid_size = 16   

# Only pick dev samples in the head of the distribution.
valid_window = 100  
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

# Number of negative examples to sample.
num_sampled = 64   

# Model Learning Rate
learning_rate = 0.01

# How many words in vocab
vocabulary_size = 5000

## TensorFlow Placeholders and Constants

In [29]:
tf.reset_default_graph()

# Input data.
train_inputs = tf.placeholder(tf.int32, shape=[None])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

### Variables

In [30]:
# Look up embeddings for inputs.
init_embeds = tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0)
embeddings = tf.Variable(init_embeds)

In [31]:
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

### NCE Loss

In [32]:
# Construct the variables for the NCE loss
nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],stddev=1.0 / np.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

In [33]:
# Compute the average NCE loss for the batch.
# tf.nce_loss automatically draws a new sample of the negative labels each
# time we evaluate the loss.
loss = tf.reduce_mean(
    tf.nn.nce_loss(nce_weights, nce_biases, train_labels, embed,
                   num_sampled, vocabulary_size))

### Optimizer

In [34]:
# Construct the Adam optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=1.0)
trainer = optimizer.minimize(loss)

In [35]:
# Compute the cosine similarity between minibatch examples and all embeddings.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings),
                             axis=1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings,
                       normalized_embeddings,
                       transpose_b=True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [36]:
# Add variable initializer.
init = tf.global_variables_initializer()

# Session

In [37]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)

In [38]:
# Usually needs to be quite large to get good results, 
# training takes a long time!
num_steps = 20001

with tf.Session(config=tf.ConfigProto(
    gpu_options=gpu_options)) as sess:
    sess.run(init)
    average_loss = 0
    for step in range(num_steps):
         
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

        # We perform one update step by evaluating the training op (including it
        # in the list of returned values for session.run()
        empty, loss_val = sess.run([trainer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 1000 == 0:
            if step > 0:
                average_loss /= 1000
            # The average loss is an estimate of the loss over the last 1000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

       

    final_embeddings = normalized_embeddings.eval()

Average loss at step  0 :  208.69786071777344


KeyboardInterrupt: 

# Visualizing Results

In [ ]:
!pip install python-bidi

In [ ]:
!pip install arabic_reshaper

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from bidi.algorithm import get_display
import arabic_reshaper

In [ ]:
def plot_with_labels(low_dim_embs, labels):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  #in inches
    for i, label in enumerate(labels):
        reshaped_text = arabic_reshaper.reshape(label)
        artext = get_display(reshaped_text)

        x, y = low_dim_embs[i,:]
        plt.scatter(x, y)
        plt.annotate(artext,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

## TSNE

* https://lvdmaaten.github.io/tsne/
* https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding

Dimensionality reduction to 2-D vectors (down from 150), this takes awhile.

In [ ]:
type(final_embeddings)

In [ ]:
final_embeddings.shape

In [ ]:
from  sklearn.manifold import TSNE
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)

In [ ]:
plot_only = 2000
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only,:])

In [ ]:
labels = [vocabulary[i] for i in range(plot_only)]

In [ ]:
low_dim_embs.shape

In [ ]:
low_dim_embs[0]

In [ ]:
labels[2]

In [ ]:
plot_with_labels(low_dim_embs, labels)

In [ ]:
plot_with_labels(low_dim_embs, labels)
plt.xlim(-10,10)
plt.ylim(-10,10)

In [ ]:
labels.index('سازد')

In [ ]:
labels.index('لباس')

In [ ]:
low_dim_embs[labels.index('سازد')]

In [ ]:
low_dim_embs[labels.index('لباس')]

In [ ]:
low_dim_embs[labels.index('سازد')]

In [ ]:
np.abs(np.subtract(low_dim_embs,low_dim_embs[labels.index('سازد')]))

In [ ]:
np.subtract(low_dim_embs,low_dim_embs[labels.index('سازد')])

In [ ]:
np.abs(np.subtract(low_dim_embs,low_dim_embs[labels.index('روزه')])).argmin()

In [ ]:
a = np.abs(np.subtract(low_dim_embs,low_dim_embs[labels.index('هستید')]))
b = np.array(list(map(lambda x : np.sqrt(x[0]**2+x[1]**2),a)))
print(b)
print(b.min())
labels[b.argmin()]

In [43]:
temp_array = low_dim_embs.copy()

temp_array[labels.index('کتمان')] =[1000,1000]
print(temp_array[labels.index('کتمان')])
print(low_dim_embs[labels.index('کتمان')])
a = np.abs(np.subtract(temp_array,low_dim_embs[labels.index('توان')]))
b = np.array(list(map(lambda x : np.sqrt(x[0]**2+x[1]**2),a)))
print(b)
print(b.min())
labels[b.argmin()]



[1000. 1000.]
[-0.97312397  8.135084  ]
[12.16001444 17.19351332 15.58994136 ... 19.68070713 19.06786681
 12.32888812]
0.0


'توان'

In [47]:
low_dim_embs

array([[ -1.150194 ,  -4.066698 ],
       [ -2.512885 ,  -9.113773 ],
       [ -3.6042297,  -7.4287353],
       ...,
       [-17.750027 ,  -3.6161294],
       [  9.383962 ,  -7.291044 ],
       [  0.9433116,  -3.9239182]], dtype=float32)

In [48]:
def find_nearest(low_dim_embs, string,labels):
    def diffrence(a,b):
        return np.sqrt(np.power((a[0]-b[0]),2),np.power((a[1]-b[1]),2))
    
    print( low_dim_embs[labels.index(string)])
    idx = np.searchsorted(low_dim_embs, np.array(low_dim_embs[labels.index(string)]), side="left")
    return labels[idx]


In [49]:
find_nearest(low_dim_embs, 'لباس',labels)

[13.02263    -0.45398244]


ValueError: object too deep for desired array

## Also check out gensim!

https://radimrehurek.com/gensim/tutorial.html

https://stackoverflow.com/questions/40074412/word2vec-get-nearest-words

In [ ]:
np.save('final_embeddings_pr',final_embeddings)

In [ ]:
np.save('low_dim_embs_pr',low_dim_embs)

In [ ]:
np.save('labels_pr',labels)

In [39]:
low_dim_embs = np.load('low_dim_embs_pr.npy')

In [40]:
final_embeddings = np.load('final_embeddings_pr.npy')

In [41]:
labels = list(np.load('labels_pr.npy'))

In [42]:
print(final_embeddings)

[[ 0.13637935  0.15279537 -0.04630525 ... -0.0112639   0.07838436
  -0.04204721]
 [ 0.08898304 -0.00348179 -0.09276596 ... -0.05182479 -0.03693384
   0.12826647]
 [-0.05376651 -0.00978381 -0.04858399 ...  0.00134597 -0.09163094
   0.04494855]
 ...
 [ 0.09388983 -0.11427605 -0.13962652 ... -0.02901328  0.03078637
   0.00666006]
 [ 0.09084063 -0.02895664 -0.04087808 ...  0.13614091 -0.10895679
   0.02612192]
 [ 0.0786558   0.09809073 -0.04009677 ...  0.06733087 -0.00757584
  -0.1196432 ]]
